In [1]:
# Get credentials
from IPython.utils import io
with io.capture_output() as captured:
    %run ../Introduction.ipynb


# The `Sentenai` Client

Sentenai repositories all have a standard REST API that is exposed in Python through the `Sentenai` class. This class takes the host and port of the Sentenai repository you're trying to connect to.

By default the Sentenai class tries to connect to `http://localhost:3333` if not `host` or `port` argument is provided.

In [2]:
# Instantiate client
from datetime import datetime
from sentenai import Sentenai

sentenai = Sentenai(host=host, port=port)

When you instantiate a `Sentenai` object, it will connect to the remote repository, returning an error if the connection cannot be made.

## Stream Databases

Every Sentenai repository contains a set of uniquely named stream databases. A stream database is a tree structured database of streams, connected by parent-child relationships. A `Stream` in Sentenai is analogous to a typed column in a relational database, and a group of children with the same parent node is equivalent to a table in a relational database. Instead of rows with primary keys, streams are related to each other by the time intervals over which each stream's values are valid.

To initialize a stream database, we must choose its `origin`. An origin is the point of time at which this database starts. No data may be stored for intervals that precede this time. The default `origin` in Sentenai is the Unix Epoch. Sentenai represents time in nanoseconds, so a stream database initialized with the default `origin` will be able to represent time itervals between 1970 and ~2530.

In addition to specifying the origin as a point in time, we can also choose to explicity specify a database be instantiated with no origin, by passing `None`. A stream with `no` origin uses nanosecond offsets from 0 instead of timestamps. This can be useful for data where the time reference is unknown, such as simulated data or predicted data.

In [3]:
sentenai.init('test-db')

Database(Sentenai(host="http://storm:7280"), "test-db")

A newly initialized stream will have no information except its origin. We can look at the list of dbs in a repository by treating the `Sentenai` object as dict-like:

In [4]:
for db in sentenai:
    print(db)

.apps
acft
acft2
met
q
synth
test
test-2
test-db
testDB1
testDBS


Just as with a dictionary, we can access a db:

In [5]:
test = sentenai['test-db']

And we can delete a db via Python's `del` keyword:

In [6]:
del sentenai['test-db']

This action can be reverted, though the functionality is currently not exposed in the Python API.

For more information about working with stream databases, continue on to [Stream Databases](Databases.ipynb).

## TSPL

Sentenai provides generalized temporal data fusion capabilities through its TSPL stream processing language. This language is designed to be used by analysists and domain experts to model complex behavior across multiple sources of data.

TSPL's syntax is SQL-like, in that many functions will perform similarly to SQL functions with the same name, but unlike SQL, TSPL is temporal rather than relational. That means that while there is some overlap in functionality, there are broad sets of transformations that you can do in TSPL but not SQL, and vice-versa.

Since TSPL is central to working with data in Sentenai, executing TSPL is done directly through the `Sentenai` object:

In [7]:
sentenai('True when frequency(30s)')

'True when frequency(30s)'

Executing this simple program can be done using python's slice functionality.

In [8]:
sentenai('True when frequency(30s) origin 2015-01-01T00:00')[datetime(2015,1,1) : datetime(2015,1,1,1,1) : 5]

[{'end': numpy.datetime64('2015-01-01T00:00:30.000000000'),
  'start': numpy.datetime64('2015-01-01T00:00:00.000000000'),
  'value': True},
 {'end': numpy.datetime64('2015-01-01T00:01:00.000000000'),
  'start': numpy.datetime64('2015-01-01T00:00:30.000000000'),
  'value': True},
 {'end': numpy.datetime64('2015-01-01T00:01:30.000000000'),
  'start': numpy.datetime64('2015-01-01T00:01:00.000000000'),
  'value': True},
 {'end': numpy.datetime64('2015-01-01T00:02:00.000000000'),
  'start': numpy.datetime64('2015-01-01T00:01:30.000000000'),
  'value': True},
 {'end': numpy.datetime64('2015-01-01T00:02:30.000000000'),
  'start': numpy.datetime64('2015-01-01T00:02:00.000000000'),
  'value': True}]

This prgram defined a stream of boolean values starting at origin `2015-01-01T00:00`, occuring on 30 second intervals.

Ths slicing defines the start time of the computation, the stop time of the computation, and the maximum number of results to return.

We also can easily output this result to a dataframe using the `df` method on the `Sentenai` object:

In [9]:
sentenai.df('True when frequency(30s) origin 2015-01-01T00:00')[datetime(2015,1,1) : datetime(2015,1,1,1,1) : 5]

,start,end,duration,value
0,2015-01-01 00:00:00,2015-01-01 00:00:30,0 days 00:00:30,True
1,2015-01-01 00:00:30,2015-01-01 00:01:00,0 days 00:00:30,True
2,2015-01-01 00:01:00,2015-01-01 00:01:30,0 days 00:00:30,True
3,2015-01-01 00:01:30,2015-01-01 00:02:00,0 days 00:00:30,True
4,2015-01-01 00:02:00,2015-01-01 00:02:30,0 days 00:00:30,True


For more information about writing and executing TSPL, continue on to [TSPL](TSPL.ipynb)